# YOLO CROP THE PERSONS FROM THE DATASET

In [ ]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO

def get_yolo_preds_and_save_crops(model, input_img_path, cropped_folder, no_person_folder, confidence_threshold=0.6):
    # Ensure output directories exist
    if not os.path.exists(cropped_folder):
        os.makedirs(cropped_folder)

    if not os.path.exists(no_person_folder):
        os.makedirs(no_person_folder)


    # Read the input image
    image = cv2.imread(input_img_path)
    if image is None:
        print(f"Error: Unable to load image {input_img_path}")
        return

    H, W = image.shape[:2]

    # Run YOLOv3 inference
    results = model.predict(input_img_path, device="cuda", conf=confidence_threshold)

    # Extract detected objects
    boxes = []
    confidences = []

    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0].item())  # Get class ID
            confidence = box.conf[0].item()  # Confidence score
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            
                  # Ensure coordinates are within image bounds
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(W, x2), min(H, y2)
            
            if confidence > confidence_threshold and class_id == 0:  # Class ID 0 is "person"
                boxes.append([x1, y1, x2 - x1, y2 - y1])  # Convert to (x, y, w, h)
                confidences.append(confidence)

    # If no person detected, save original image
    if not boxes:
        no_person_filename = os.path.join(no_person_folder, os.path.basename(input_img_path))
        cv2.imwrite(no_person_filename, image)
        print(f"Saved image with no person detected: {no_person_filename}")
        return  

    # Save cropped persons
    for i, (x, y, w, h) in enumerate(boxes):
        cropped_person = image[y:y + h, x:x + w]

        if cropped_person.size == 0:
            print(f"Warning: Empty crop for {input_img_path} at box {boxes[i]}")
            continue

        original_filename = os.path.splitext(os.path.basename(input_img_path))[0]
        cropped_filename = os.path.join(cropped_folder, f"{original_filename}_person_{i + 1}_conf_{confidences[i]:.2f}.jpg")
        cv2.imwrite(cropped_filename, cropped_person)
        print(f"Saved cropped person: {cropped_filename}")

def process_images_in_folder(model, input_folder, output_folder, no_person_folder, confidence_threshold=0.5):
    for label in ['Appropriate', 'Inappropriate']:  # Iterate over labeled folders
        label_folder = os.path.join(input_folder, label)
        output_label_folder = os.path.join(output_folder, label)

        os.makedirs(output_label_folder, exist_ok=True)

        # Process each image in the folder
        for img_filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, img_filename)

            if os.path.isfile(img_path) and img_filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                print(f"Processing image: {img_path}")
                get_yolo_preds_and_save_crops(model, img_path, output_label_folder, no_person_folder, confidence_threshold)

# Load Ultralytics YOLOv3 model with CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO("yolov3u.pt").to(device)

print(f"YOLO model is set to use: {device.upper()}")

# Example usage
input_folder = "./augmented_dataset"
output_folder = "./cropped_full"
no_person_folder = "./no_person_full"

process_images_in_folder(model, input_folder, output_folder, no_person_folder, confidence_threshold=0.6)


# takes 4 mins only

In [ ]:
import os
import cv2

def augment_dataset(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, filename)

                # Load image
                image = cv2.imread(file_path)
                if image is None:
                    print(f"Error reading image {file_path}")
                    continue

                # Convert original to grayscale
                # gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Flip the image horizontally
                flipped_image = cv2.flip(image, 1)

                # Save augmented images
                base_name, ext = os.path.splitext(filename)
                output_label_folder = os.path.join(output_folder, os.path.basename(root))
                os.makedirs(output_label_folder, exist_ok=True)

                # Paths for saving
                original_path = os.path.join(output_label_folder, f"{base_name}_original{ext}")
                # original_gray_path = os.path.join(output_label_folder, f"{base_name}_gray{ext}")
                flipped_color_path = os.path.join(output_label_folder, f"{base_name}_flipped{ext}")

                # Save the images
                cv2.imwrite(original_path, image)
                # cv2.imwrite(original_gray_path, gray_image)
                cv2.imwrite(flipped_color_path, flipped_image)

                print(f"Saved {original_path}, and {flipped_color_path}")

# Example usage
input_folder = "./Full Dataset"  # Path to your input dataset
output_folder = "./augmented_dataset"  # Path to save the augmented dataset

augment_dataset(input_folder, output_folder)


Saved ./augmented_dataset\Appropriate\00000001_original.jpg, ./augmented_dataset\Appropriate\00000001_gray.jpg, and ./augmented_dataset\Appropriate\00000001_flipped.jpg
Saved ./augmented_dataset\Appropriate\00000002_original.jpg, ./augmented_dataset\Appropriate\00000002_gray.jpg, and ./augmented_dataset\Appropriate\00000002_flipped.jpg
Saved ./augmented_dataset\Appropriate\0000000296_original.png, ./augmented_dataset\Appropriate\0000000296_gray.png, and ./augmented_dataset\Appropriate\0000000296_flipped.png
Saved ./augmented_dataset\Appropriate\00000003_original.jpg, ./augmented_dataset\Appropriate\00000003_gray.jpg, and ./augmented_dataset\Appropriate\00000003_flipped.jpg
Saved ./augmented_dataset\Appropriate\00000004_original.jpg, ./augmented_dataset\Appropriate\00000004_gray.jpg, and ./augmented_dataset\Appropriate\00000004_flipped.jpg
Saved ./augmented_dataset\Appropriate\00000006_original.png, ./augmented_dataset\Appropriate\00000006_gray.png, and ./augmented_dataset\Appropriate\0

# takes 18 mins

In [8]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image

def augment_dataset_cuda(input_folder, output_folder, device='cuda'):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Define transformations
    to_tensor = transforms.ToTensor()
    to_pil = transforms.ToPILImage()
    grayscale = transforms.Grayscale(num_output_channels=3)
    horizontal_flip = transforms.RandomHorizontalFlip(p=1.0)  # Always flip

    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, filename)
                
                try:
                    # Load image and move it to the GPU
                    image = Image.open(file_path).convert('RGB')
                    image_tensor = to_tensor(image).to(device)

                    # Convert original to grayscale
                    gray_image_tensor = grayscale(image_tensor).to(device)
                    
                    # Flip the image horizontally
                    flipped_image_tensor = horizontal_flip(image_tensor).to(device)

                    # Convert tensors back to PIL images
                    original_image = to_pil(image_tensor.cpu())
                    gray_image = to_pil(gray_image_tensor.cpu())
                    flipped_image = to_pil(flipped_image_tensor.cpu())

                    # Prepare save paths
                    base_name, ext = os.path.splitext(filename)
                    output_label_folder = os.path.join(output_folder, os.path.basename(root))
                    os.makedirs(output_label_folder, exist_ok=True)

                    # Paths for saving
                    original_path = os.path.join(output_label_folder, f"{base_name}_original{ext}")
                    original_gray_path = os.path.join(output_label_folder, f"{base_name}_gray{ext}")
                    flipped_color_path = os.path.join(output_label_folder, f"{base_name}_flipped{ext}")

                    # Save the images
                    original_image.save(original_path)
                    gray_image.save(original_gray_path)
                    flipped_image.save(flipped_color_path)

                    print(f"Saved {original_path}, {original_gray_path}, and {flipped_color_path}")

                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Example usage
input_folder = "./Full Dataset"  # Path to your input dataset
output_folder = "./augmented_datasetv2"

# Move computations to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

augment_dataset_cuda(input_folder, output_folder, device)


Saved ./augmented_datasetv2\Appropriate\00000001_original.jpg, ./augmented_datasetv2\Appropriate\00000001_gray.jpg, and ./augmented_datasetv2\Appropriate\00000001_flipped.jpg
Saved ./augmented_datasetv2\Appropriate\00000002_original.jpg, ./augmented_datasetv2\Appropriate\00000002_gray.jpg, and ./augmented_datasetv2\Appropriate\00000002_flipped.jpg
Saved ./augmented_datasetv2\Appropriate\0000000296_original.png, ./augmented_datasetv2\Appropriate\0000000296_gray.png, and ./augmented_datasetv2\Appropriate\0000000296_flipped.png
Saved ./augmented_datasetv2\Appropriate\00000003_original.jpg, ./augmented_datasetv2\Appropriate\00000003_gray.jpg, and ./augmented_datasetv2\Appropriate\00000003_flipped.jpg
Saved ./augmented_datasetv2\Appropriate\00000004_original.jpg, ./augmented_datasetv2\Appropriate\00000004_gray.jpg, and ./augmented_datasetv2\Appropriate\00000004_flipped.jpg
Saved ./augmented_datasetv2\Appropriate\00000006_original.png, ./augmented_datasetv2\Appropriate\00000006_gray.png, and

E:\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Saved ./augmented_datasetv2\Appropriate\ia_100000010552_original.png, ./augmented_datasetv2\Appropriate\ia_100000010552_gray.png, and ./augmented_datasetv2\Appropriate\ia_100000010552_flipped.png
Saved ./augmented_datasetv2\Appropriate\ia_100000010562_original.jpg, ./augmented_datasetv2\Appropriate\ia_100000010562_gray.jpg, and ./augmented_datasetv2\Appropriate\ia_100000010562_flipped.jpg
Saved ./augmented_datasetv2\Appropriate\ia_100000010566_original.jpeg, ./augmented_datasetv2\Appropriate\ia_100000010566_gray.jpeg, and ./augmented_datasetv2\Appropriate\ia_100000010566_flipped.jpeg
Saved ./augmented_datasetv2\Appropriate\ia_100000010571_original.jpg, ./augmented_datasetv2\Appropriate\ia_100000010571_gray.jpg, and ./augmented_datasetv2\Appropriate\ia_100000010571_flipped.jpg
Saved ./augmented_datasetv2\Appropriate\ia_100000010576_original.jpg, ./augmented_datasetv2\Appropriate\ia_100000010576_gray.jpg, and ./augmented_datasetv2\Appropriate\ia_100000010576_flipped.jpg
Saved ./augmented